## Differential expression analysis control

As a way of quantifying how similar/different the expression profiles associated with different mutation patterns are, we want to count the number of differentially expressed genes between them. To make sure this makes sense, we first want to take some cancer subtypes we know are quite different, and compare the number of DE genes between them to the number of DE genes between random samples of the same size.

We expect to see that different subtypes have considerably more DE genes between them than random samples taken uniformly from the different subtypes.

In [1]:
from pathlib import Path
import pickle as pkl

import pandas as pd

import sys; sys.path.append('..')
import config as cfg

%load_ext autoreload
%autoreload 2

### Load datasets

In [2]:
# load counts data
all_counts_df = pd.read_csv(cfg.processed_counts_file, sep='\t', index_col=0)
print(all_counts_df.shape)
all_counts_df.iloc[:5, :5]

(10530, 60498)


,ENSG00000000003.14,ENSG00000000005.5,ENSG00000000419.12,ENSG00000000457.13,ENSG00000000460.16
sample_id,,,,,
TCGA-02-0047-01,2132,1,552,387,269
TCGA-02-0055-01,2590,742,2110,493,452
TCGA-02-2483-01,4438,1,2230,735,653
TCGA-02-2485-01,4615,4,1022,629,672
TCGA-04-1331-01,6159,10,2752,1173,1091


In [3]:
# load cancer types
sample_info_df = pd.read_csv(cfg.de_sample_info, sep='\t', index_col=0)
print(sample_info_df.shape)
sample_info_df.head()

(11060, 3)


,sample_type,cancer_type,id_for_stratification
sample_id,,,
TCGA-02-0047-01,Primary Solid Tumor,GBM,GBMPrimary Solid Tumor
TCGA-02-0055-01,Primary Solid Tumor,GBM,GBMPrimary Solid Tumor
TCGA-02-2483-01,Primary Solid Tumor,GBM,GBMPrimary Solid Tumor
TCGA-02-2485-01,Primary Solid Tumor,GBM,GBMPrimary Solid Tumor
TCGA-02-2486-01,Primary Solid Tumor,GBM,GBMPrimary Solid Tumor


In [4]:
# load mutation status
pancancer_pickle = Path('/home/jake/research/mpmp/data/pancancer_data.pkl')
with open(pancancer_pickle, 'rb') as f:
    pancancer_data = pkl.load(f)
    
mutation_df = pancancer_data[1]
print(mutation_df.shape)
mutation_df.iloc[:5, :5]

(9074, 20938)


,5S_rRNA,A1BG,A1CF,A2M,A2ML1
SAMPLE_BARCODE,,,,,
TCGA-02-0047-01,0,0,0,0,0
TCGA-02-0055-01,0,0,0,0,0
TCGA-02-2483-01,0,0,0,0,0
TCGA-02-2485-01,0,0,0,0,0
TCGA-02-2486-01,0,0,0,0,0


### DE between IDH1 mutant/wild-type samples in low-grade glioma

In [5]:
cfg.de_input_dir.mkdir(parents=True, exist_ok=True)
cfg.de_output_dir.mkdir(parents=True, exist_ok=True)

base_dir = str(cfg.de_base_dir)
output_dir = str(cfg.de_output_dir)

In [6]:
# get LGG samples from counts data
lgg_samples = (
    sample_info_df[sample_info_df.cancer_type == 'LGG'].index
      .intersection(all_counts_df.index)
      .intersection(mutation_df.index)
)
lgg_counts_df = all_counts_df.loc[lgg_samples, :]
print(lgg_counts_df.shape)
lgg_counts_df.iloc[:5, :5]

(500, 60498)


,ENSG00000000003.14,ENSG00000000005.5,ENSG00000000419.12,ENSG00000000457.13,ENSG00000000460.16
TCGA-CS-4938-01,5768,40,1083,677,369
TCGA-CS-4941-01,5740,91,919,980,554
TCGA-CS-4942-01,4187,0,820,804,433
TCGA-CS-4943-01,3458,2,1344,804,759
TCGA-CS-4944-01,5868,12,712,569,170


In [7]:
# save LGG samples to file, to be loaded by DESeq2
input_file = cfg.de_input_dir / 'lgg_counts.tsv'
input_str = str(input_file)

lgg_counts_df.to_csv(input_file, sep='\t')

In [8]:
# get IDH1 mutation status
idh1_status_df = (mutation_df
    .loc[lgg_samples, ['IDH1']]
    .rename(columns={'IDH1': 'group'})
)
idh1_status_df.head()

,group
TCGA-CS-4938-01,1
TCGA-CS-4941-01,0
TCGA-CS-4942-01,1
TCGA-CS-4943-01,1
TCGA-CS-4944-01,1


In [9]:
# save mutation status to file, to be loaded by DESeq2
input_metadata_file = cfg.de_input_dir / 'lgg_idh1_status.tsv'
input_metadata_str = str(input_metadata_file)

idh1_status_df.to_csv(input_metadata_file, sep='\t')

In [10]:
%load_ext rpy2.ipython

In [11]:
%%R -i base_dir -i input_metadata_str -i input_str -i output_dir

source(paste0(base_dir, '/de_analysis.R'))

get_DE_stats_DESeq(input_metadata_str,
                   input_str,
                   'LGG_IDH1',
                   output_dir)

R[write to console]: Loading required package: S4Vectors

R[write to console]: Loading required package: stats4

R[write to console]: Loading required package: BiocGenerics

R[write to console]: Loading required package: parallel

R[write to console]: 
Attaching package: ‘BiocGenerics’


R[write to console]: The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB


R[write to console]: The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


R[write to console]: The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colnames,
    dirname, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, Map, mapply, match, mget,
    order, paste, pmax, pmax.int, pmin, pmin.int, Position, 

[1] "Checking sample ordering..."
[1] TRUE
                group
TCGA-CS-4938-01     1
TCGA-CS-4941-01     0
TCGA-CS-4942-01     1
TCGA-CS-4943-01     1
TCGA-CS-4944-01     1
TCGA-CS-5390-01     1
[1] 1 0 1 1 1
Levels: 0 1


R[write to console]:   the design formula contains one or more numeric variables with integer values,
  specifying a model with increasing fold change for higher values.
  did you mean for this to be a factor? if so, first convert
  this variable to a factor using the factor() function



### DE between random samples in low-grade glioma

We do this to generate an empirical null distribution for our results in IDH1 mutants/wild-type samples.

In [12]:
# number of random samples
n_samples = 5

In [13]:
n_mutated = idh1_status_df.sum().values[0]
n_not_mutated = idh1_status_df.shape[0] - n_mutated
print(n_mutated, n_not_mutated)

387 113


In [14]:
# we can use sklearn train_test_split to partition the data randomly
import numpy as np
from sklearn.model_selection import train_test_split

for sample_ix in range(n_samples):
    _, test_ixs = train_test_split(idh1_status_df.index,
                                   test_size=n_mutated,
                                   shuffle=True,
                                   random_state=sample_ix)
    labels_df = pd.DataFrame(
        np.zeros(idh1_status_df.shape[0]).astype(int),
        index=idh1_status_df.index.copy(),
        columns=['group']
    )
    labels_df.loc[test_ixs, 'group'] = 1
    
    save_file = cfg.de_input_dir / 'lgg_idh1_random_s{}.tsv'.format(sample_ix)
    print(str(save_file))
    labels_df.to_csv(save_file, sep='\t')

/home/jake/research/mutation-fn/4_de_analysis/de_input/lgg_idh1_random_s0.tsv
/home/jake/research/mutation-fn/4_de_analysis/de_input/lgg_idh1_random_s1.tsv
/home/jake/research/mutation-fn/4_de_analysis/de_input/lgg_idh1_random_s2.tsv
/home/jake/research/mutation-fn/4_de_analysis/de_input/lgg_idh1_random_s3.tsv
/home/jake/research/mutation-fn/4_de_analysis/de_input/lgg_idh1_random_s4.tsv


In [15]:
input_metadata_dir = str(cfg.de_input_dir)

In [ ]:
%%R -i base_dir -i input_str -i n_samples -i input_metadata_dir -i output_dir

source(paste0(base_dir, '/de_analysis.R'))

for (i in 0:(n_samples-1)) {
    print(paste('Running: ', i))
    input_metadata_str <- paste(
        input_metadata_dir, '/lgg_idh1_random_s', i, '.tsv',
        sep=''
    )
    get_DE_stats_DESeq(input_metadata_str,
                       input_str,
                       paste('LGG_IDH1_random_s', i, sep=''),
                       output_dir)
}

[1] "Running:  0"
[1] "Checking sample ordering..."
[1] TRUE
                group
TCGA-CS-4938-01     1
TCGA-CS-4941-01     1
TCGA-CS-4942-01     1
TCGA-CS-4943-01     1
TCGA-CS-4944-01     1
TCGA-CS-5390-01     1
[1] 1 1 1 1 1
Levels: 0 1


R[write to console]:   the design formula contains one or more numeric variables with integer values,
  specifying a model with increasing fold change for higher values.
  did you mean for this to be a factor? if so, first convert
  this variable to a factor using the factor() function



[1] "Running:  1"
[1] "Checking sample ordering..."
[1] TRUE
                group
TCGA-CS-4938-01     1
TCGA-CS-4941-01     0
TCGA-CS-4942-01     0
TCGA-CS-4943-01     0
TCGA-CS-4944-01     1
TCGA-CS-5390-01     1
[1] 1 0 0 0 1
Levels: 0 1


R[write to console]:   the design formula contains one or more numeric variables with integer values,
  specifying a model with increasing fold change for higher values.
  did you mean for this to be a factor? if so, first convert
  this variable to a factor using the factor() function



[1] "Running:  2"
[1] "Checking sample ordering..."
[1] TRUE
                group
TCGA-CS-4938-01     1
TCGA-CS-4941-01     1
TCGA-CS-4942-01     1
TCGA-CS-4943-01     1
TCGA-CS-4944-01     1
TCGA-CS-5390-01     1
[1] 1 1 1 1 1
Levels: 0 1


R[write to console]:   the design formula contains one or more numeric variables with integer values,
  specifying a model with increasing fold change for higher values.
  did you mean for this to be a factor? if so, first convert
  this variable to a factor using the factor() function

